# **Muertes por factor de riesgo**

Tenemos un archivo csv que recopila el número de muertes por factor de riesgo y país. Vamos a tratar de entender qué factores de riesgo acumulan más muertes en qué países.

Los datos van de 1990 al 2019.

Primero vamos a observar la tabla para empezar a entenderla.

In [33]:
import altair as alt
import pandas as pd

!pip install altair_viewer

archivo_csv = '/content/number-of-deaths-by-risk-factor.csv'

datos = pd.read_csv(archivo_csv)

datos.head()

,Entity,Code,Year,Deaths - Cause: All causes - Risk: Outdoor air pollution - OWID - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: High systolic blood pressure - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Diet high in sodium - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Diet low in whole grains - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Alcohol use - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Diet low in fruits - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Unsafe water source - Sex: Both - Age: All Ages (Number),...,Deaths - Cause: All causes - Risk: High body-mass index - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Unsafe sanitation - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: No access to handwashing facility - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Drug use - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Low bone mineral density - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Vitamin A deficiency - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Child stunting - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Discontinued breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Non-exclusive breastfeeding - Sex: Both - Age: All Ages (Number),Deaths - Cause: All causes - Risk: Iron deficiency - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,3169,25633,1045,7077,356,3185,3702,...,9518,2798,4825,174,389,2016,7686,107,2216,564
1,Afghanistan,AFG,1991,3222,25872,1055,7149,364,3248,4309,...,9489,3254,5127,188,389,2056,7886,121,2501,611
2,Afghanistan,AFG,1992,3395,26309,1075,7297,376,3351,5356,...,9528,4042,5889,211,393,2100,8568,150,3053,700
3,Afghanistan,AFG,1993,3623,26961,1103,7499,389,3480,7152,...,9611,5392,7007,232,411,2316,9875,204,3726,773
4,Afghanistan,AFG,1994,3788,27658,1134,7698,399,3610,7192,...,9675,5418,7421,247,413,2665,11031,204,3833,812


# **¿Cuáles son los principales factores de riesgo?**

Alta presión arterial, fumar y la calidad del aire son los principales factores de riesgo.

¿Hay factores de riesgo que afecten más a unas regiones que a otras?

In [35]:
# Desactivar el límite de filas en Altair
alt.data_transformers.disable_max_rows()

# Excluir las columnas no relacionadas con los factores de riesgo (Entity, Code, Year)
columnas_riesgo = datos.columns.drop(['Entity', 'Code', 'Year'])

# Calcular la suma total para cada factor de riesgo
sumas_riesgo = datos[columnas_riesgo].sum().reset_index()
sumas_riesgo.columns = ['Risk Factor', 'Total Deaths']

# Crear la visualización en Altair
chart = alt.Chart(sumas_riesgo).mark_bar().encode(
    x=alt.X('Total Deaths:Q', title='Total de Muertes'),
    y=alt.Y('Risk Factor:N', title='Factor de Riesgo', sort='-x'),  # Ordenar por el número de muertes
    color='Risk Factor:N',
    tooltip=['Risk Factor', 'Total Deaths']
).properties(
    title='Total de Muertes por Factor de Riesgo'
)

chart

alt.Chart(...)

# **Malos humos: ¿Qué países sufren más muertes por polución y tabaco?**

China e India encabezan el listado de los países con más muertes por cuestiones respiratorias. Pero realmente son los dos países más poblados del mundo, aunque no por tanta diferencia. China tiene 4 veces más población que Estados Unidos, pero registra 8 veces más muertes por estas causas. Además China tiene casi la misma población que India, lo que nos lleva a pensar que sí podrían tener un problema histórico con la calidad del aire.


In [36]:
# Palabras clave para identificar regiones o grupos de países
palabras_clave_region = ['Region', 'World', 'Bank', 'WHO', 'WB', 'G20', 'OECD Countries']

# Filtrar y preparar los datos
causas_no_naturales_cols = [col for col in datos.columns if "pollution" in col.lower() or "smoking" in col.lower()]
causas_no_naturales_cols.insert(0, "Entity")
causas_no_naturales_cols.insert(1, "Year")
data_no_naturales = datos[causas_no_naturales_cols]

# Filtrar para incluir solo países
data_paises = data_no_naturales[~data_no_naturales['Entity'].str.contains('|'.join(palabras_clave_region))]

muertes_por_pais = data_paises.groupby("Entity").sum().reset_index()
muertes_por_pais["Total Muertes No Naturales"] = muertes_por_pais.drop(columns=["Entity", "Year"]).sum(axis=1)
paises_con_mas_muertes = muertes_por_pais[["Entity", "Total Muertes No Naturales"]].sort_values(by="Total Muertes No Naturales", ascending=False)

# Crear la visualización en Altair
chart = alt.Chart(paises_con_mas_muertes.head(10)).mark_bar().encode(
    x='Total Muertes No Naturales',
    y=alt.Y('Entity', sort='-x'),
    color='Entity',
    tooltip=['Entity', 'Total Muertes No Naturales']
).properties(
    title='Top 10 Países con más Muertes por Causas No Naturales'
)

chart

alt.Chart(...)

# **A punto de explotar: ¿Qué países tienen más problemas con la presión arterial?**

China de nuevo encabeza con demasiada diferencia el número de muertes esta vez por alta presión arterial. Prácticamente duplica al siguiente en la lista que es India teniendo casi la misma población.

Aunque Estados Unidos tiene una afección per cápita mayor que China.

In [38]:
# Palabras clave para identificar regiones o grupos de países
palabras_clave_region = ['Region', 'World', 'Bank', 'WHO', 'WB', 'G20', 'OECD Countries']

# Filtrar para incluir solo países
data_paises = datos[~datos['Entity'].str.contains('|'.join(palabras_clave_region))]

# Asumiendo que la columna de interés se llama 'Muertes por Alta Presión Arterial'
columna_presion_arterial = 'Deaths - Cause: All causes - Risk: High systolic blood pressure - Sex: Both - Age: All Ages (Number)'  # Ajusta este nombre según tu conjunto de datos

# Seleccionar y sumar las muertes por alta presión arterial para cada país
muertes_presion_arterial = data_paises.groupby('Entity')[columna_presion_arterial].sum().reset_index()
muertes_presion_arterial = muertes_presion_arterial.sort_values(by=columna_presion_arterial, ascending=False)

# Crear la visualización en Altair
chart = alt.Chart(muertes_presion_arterial.head(10)).mark_bar().encode(
    x=alt.X(columna_presion_arterial + ':Q', title='Total de Muertes por Alta Presión Arterial'),
    y=alt.Y('Entity:N', title='País', sort='-x'),
    color='Entity:N',
    tooltip=['Entity', columna_presion_arterial]
).properties(
    title='Top 10 Países con más Muertes por Alta Presión Arterial'
)

chart

alt.Chart(...)

# **Conclusión: Cuídense mucho**

Pero cuiden también el planeta.

La presión arterial es multifactorial, pero el caso de China podría estar ligado con la calidad del aire.